In [1]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn import metrics, model_selection
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup


tokenizers.__version__: 0.11.6
transformers.__version__: 4.17.0


In [2]:
ROOT = '../input/nbme-score-clinical-patient-notes'
MODEL_NAME = '../input/robertalarge/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf'
#BATCH_SIZE = 16
BATCH_SIZE = 4
N_FOLDS = 5
#N_FOLDS = 10
trn_folds=[0,1]
MODELS_PATH = '../input/robertalarge5best/roberta-4fold-models'

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [4]:
def create_test_df():
    feats = pd.read_csv(f"{ROOT}/features.csv")
    notes = pd.read_csv(f"{ROOT}/patient_notes.csv")
    test = pd.read_csv(f"{ROOT}/test.csv")

    merged = test.merge(notes, how = "left")
    merged = merged.merge(feats, how = "left")

    def process_feature_text(text):
        return text.replace('-OR-',' or ').replace('-',' ')
    def clean_backslash_chars(text):
        return text.replace('\n',' ').replace('\s',' ').replace('\t',' ')
    
    merged["feature_text"] = [process_feature_text(x) for x in merged["feature_text"]]
    merged['pn_history']=merged['pn_history'].apply(lambda x: x.strip()).apply(clean_backslash_chars)
    
    print(merged.shape)
    return merged

In [5]:
class NBMETestData(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        example = self.data.loc[idx]
        tokenized = self.tokenizer(
            example["feature_text"],
            example["pn_history"],
            truncation = "only_second",
            max_length = 416, ##max-length
            padding = "max_length",
            return_offsets_mapping = True
        )
        tokenized["sequence_ids"] = tokenized.sequence_ids()

        input_ids = np.array(tokenized["input_ids"])
        attention_mask = np.array(tokenized["attention_mask"])
        offset_mapping = np.array(tokenized["offset_mapping"])
        sequence_ids = np.array(tokenized["sequence_ids"]).astype("float16")

        return input_ids, attention_mask, offset_mapping, sequence_ids

In [6]:
class NBMEModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(MODEL_NAME)
        self.config = AutoConfig.from_pretrained(MODEL_NAME)
#         self.backbone = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
#         self.config = AutoConfig.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
        self.dropout = torch.nn.Dropout(p=0.2)
        self.classifier = torch.nn.Linear(self.config.hidden_size,512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 1)
        
    def forward(self, input_ids, attention_mask):
        pooler_outputs = self.backbone(input_ids=input_ids, 
                                       attention_mask=attention_mask)[0]
        #logits = self.classifier(self.dropout(pooler_outputs)).squeeze(-1)
        logits = self.classifier(self.dropout(pooler_outputs)).squeeze(-1)
        logits = torch.nn.ReLU()(logits)
        #logits = self.fc2(self.dropout(logits))
        logits = self.fc3(self.dropout(logits)).squeeze(-1)
        return logits

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def get_location_predictions(preds, offset_mapping, sequence_ids, test = False):
    all_predictions = []
    for pred, offsets, seq_ids in zip(preds, offset_mapping, sequence_ids):
        pred = sigmoid(pred)
        start_idx = None
        current_preds = []
        for p, o, s_id in zip(pred, offsets, seq_ids):
            if s_id is None or s_id == 0:
                continue
            if p > 0.5:
                if start_idx is None:
                    start_idx = o[0]
                end_idx = o[1]
            elif start_idx is not None:
                if test:
                    current_preds.append(f"{start_idx} {end_idx}")
                else:
                    current_preds.append((start_idx, end_idx))
                start_idx = None
        if test:
            all_predictions.append("; ".join(current_preds))
        else:
            all_predictions.append(current_preds)
    return all_predictions


In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

test = create_test_df()
test_ds = NBMETestData(test, tokenizer)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, pin_memory=True, 
                                      shuffle=False, drop_last=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(5, 6)


In [9]:
all_preds = None
offsets = []
seq_ids = []

for fold in trn_folds:
    model = NBMEModel().to(DEVICE)
    model.load_state_dict(torch.load(f'{MODELS_PATH}/nbme_{fold}.pth', map_location=DEVICE))
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in tqdm(test_dl):
            input_ids = batch[0].to(DEVICE)
            attention_mask = batch[1].to(DEVICE)
            logits = model(input_ids, attention_mask)
            preds.append(logits.cpu().numpy())
            if fold == 0:                  # only in the first fold
                offset_mapping = batch[2]
                sequence_ids = batch[3]
                offsets.append(offset_mapping.numpy())
                seq_ids.append(sequence_ids.numpy())
    preds = np.concatenate(preds, axis=0)
    if all_preds is None:
        all_preds = np.array(preds).astype(np.float32)
    else:
        all_preds += np.array(preds).astype(np.float32)
    torch.cuda.empty_cache()
    
    
all_preds /= N_FOLDS
all_preds = all_preds.squeeze()

offsets = np.concatenate(offsets, axis=0)
seq_ids = np.concatenate(seq_ids, axis=0)

print(all_preds.shape, offsets.shape, seq_ids.shape)


Some weights of the model checkpoint at ../input/robertalarge/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at ../input/robertalarge/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/2 [00:00<?, ?it/s]

(5, 416) (5, 416, 2) (5, 416)


In [10]:
location_preds = get_location_predictions(all_preds, offsets, seq_ids, test=True)

test["location"] = location_preds
test[["id", "location"]].to_csv("submission.csv", index = False)
pd.read_csv("submission.csv").head()

,id,location
0,00016_000,696 724
1,00016_001,668 693
2,00016_002,203 217
3,00016_003,70 91
4,00016_004,222 258
